In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
from matplotlib import rcParams
import sympy as sy
import scipy.constants as pc
from sympy import *
from math import log10, floor

rcParams['font.size'] = 16
rcParams['font.weight'] = 'bold'
rcParams['text.usetex'] = True
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Helvetica']

In [3]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)], zMaxArray[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (koeffizienten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def einfachGaussMultMax(unsicherheiten, werte, exponenten):
    max_a = 0
    num = -1
    for i in range (unsicherheiten.size):
        maxi = (exponenten[i]*unsicherheiten[i]/werte[i])**2
        if max_a <= maxi:
            max_a = maxi
            num = i
    return (max_a, num)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]-(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))

def round_to_j(x,j):
    return round(x, (j-1)-int(floor(log10(abs(x)))))

def print_to_table(avgs, stds, rows, cols,
                   highlight_best=True,
                   max_is_best=True,
                   avg_format='{0:!>4.1f}',
                   std_format='{1:!<4.1f}',
                   file=None,
                   col_orientation='c'):
    # Find best values
    best_avg = {}
    if highlight_best:
        if max_is_best:
            for c in cols:
                idx = np.argmax(avgs[c], axis=0)
                best_avg[c] = idx
        else:
            for c in cols:
                idx = np.argmin(avgs[c], axis=0)
                best_avg[c] = idx

    # Header
    print_str = '\\begin{tabular}{l' + (" "+col_orientation) * len(cols) + '} \hline\n'
    for c in cols:
        print_str += ' & ' + c
    print_str += ' \\\\ \n \hline'

    # Content
    for m in range(len(rows)):
        row_name = rows[m]
        print_str += '\n{0}'.format(row_name)
        for c in cols:
            print_str += ' &\n '
            print_list = [avgs[c][m]]
            print_list += [stds[c][m]]
            if m == best_avg.get(c, None):
                print_str += ('$\mathbf{{'+ avg_format + ' \\pm ' + std_format + '}}$').format(*print_list)
            else:
                print_str += ('$'+ avg_format + ' \\pm ' + std_format + '$').format(*print_list)

        print_str += "\n \\vspace*{0.0mm} \\\\"
        print_str = print_str.replace('!', '\\phantom{0}')

    # Footer
    print_str += '\n \hline \n \end{tabular}'

    # Write output
    if file is None:
        print(print_str)
    else:
        with open(file, 'w') as f:
            f.write(print_str)

In [4]:
# # Standard plotting

# fig, ax = plt.subplots(figsize=(10,6))
# ax.tick_params(axis='both', which='major', labelsize=18)
# ax.tick_params(axis='both', which='minor', labelsize=16)

# #ax.plot(x, y, color='b', label=r'label')
# #ax.scatter(x, y, color='r', label=r'label')
# #ax.errorbar(am_x, am_y, xerr=std_x, yerr=std_y, fmt='D', color='#d0740b', label=r'label')

# ax.set(xlabel=r'x', ylabel=r'y', title=r'Titel')
# ax.legend()
# ax.grid()

# #fig.savefig("test.png")

In [17]:
#Fehler fort B2 G

expo1 = np.array([1,-1])
werte1 = np.array([35,5])
unsic1 = np.array([0.5,0.5])

print(werte1[0]/werte1[1] * einfachGaussMult(unsic1, werte1, expo1))


0.707106781186548


In [18]:
#Fehler fort B2 G

expo2 = np.array([1,-1])
werte2 = np.array([34,5])
unsic2 = np.array([0.5,0.5])

print(werte2[0]/werte2[1] * einfachGaussMult(unsic2, werte2, expo2))

0.687313611097583


In [19]:
#Fehler fort vges ß

Vges = np.array([werte1[0]/werte1[1], werte2[0]/werte2[1]])
unsic = np.array([0.182002747232013, 0.177200451466694])

for i in range(2):
    expo3 = np.array([1,-1])
    werte3 = np.array([Vges[i],19.1])
    unsic3 = np.array([unsic[i],0.7])

    print("für vges",[i], werte3[0]/werte3[1] * einfachGaussMult(unsic3, werte3, expo3))

für vges [0] 0.0164684515082028
für vges [1] 0.0160099838134359


In [29]:
#Fehler fort B ß
expo4 = np.array([1,-1])
werte4 = np.array([26,werte1[0]/werte1[1]])
unsic4 = np.array([1,werte1[0]/werte1[1] * einfachGaussMult(unsic1, werte1, expo1)])

print("G1",werte4[0]/werte4[1])
print("G1_f", werte4[0]/werte4[1] * einfachGaussMult(unsic4, werte4, expo4))

expo5 = np.array([1,-1])
werte5 = np.array([25,werte2[0]/werte2[1]])
unsic5 = np.array([1,werte2[0]/werte2[1] * einfachGaussMult(unsic2, werte2, expo2)])

print("G2",werte5[0]/werte5[1])
print("G2_f", werte5[0]/werte5[1] * einfachGaussMult(unsic5, werte5, expo5))

G1 3.7142857142857144
G1_f 0.401475828018490
G2 3.6764705882352944
G2_f 0.399642042467673


In [30]:
#Fehler fort D ß
expo4 = np.array([1,-1])
werte4 = np.array([8,werte1[0]/werte1[1]])  #DAs gleiche D, ß1
unsic4 = np.array([1,werte1[0]/werte1[1] * einfachGaussMult(unsic1, werte1, expo1)])

print("G1",werte4[0]/werte4[1])
print("G1_f", werte4[0]/werte4[1] * einfachGaussMult(unsic4, werte4, expo4))

expo5 = np.array([1,-1])
werte5 = np.array([8,werte2[0]/werte2[1]]) #Das gleiche D aber ß2
unsic5 = np.array([1,werte2[0]/werte2[1] * einfachGaussMult(unsic2, werte2, expo2)])

print("G2",werte5[0]/werte5[1])
print("G2_f", werte5[0]/werte5[1] * einfachGaussMult(unsic5, werte5, expo5))

G1 1.1428571428571428
G1_f 0.183673469387755
G2 1.1764705882352942
G2_f 0.189120210304754


In [20]:
print(einfachGaussMultMax(unsic1, werte1, expo1))
print(einfachGaussMultMax(unsic2, werte2, expo2))

for i in range(2):
    expo3 = np.array([1,-1])
    werte3 = np.array([Vges[i],19.1])
    unsic3 = np.array([unsic[i],0.7])

    print("für vges",[i], einfachGaussMultMax(unsic3, werte3, expo3))

(0.010000000000000002, 1)
(0.010000000000000002, 1)
für vges [0] (0.001343164935171733, 1)
für vges [1] (0.001343164935171733, 1)
